In [28]:
import graphistry
import graphistry.constants as config
import pandas as pd
import umap.plot

import os
from collections import Counter
from importlib import reload
import matplotlib.pylab as plt
import numpy as np

from data import get_botnet_dataframe

import warnings
warnings.filterwarnings('ignore')

In [2]:
graphistry.register(api=3, protocol="https", server="hub.graphistry.com", username="...", password="...") 

# We import malware dataset


In [3]:
edf = get_botnet_dataframe(150000)
edf = edf.sample(1500)

[__init__.py:40 - get_botnet_dataframe() ]
   Loading Botnet Capture Data with Targets



In [4]:
edf = edf.drop_duplicates()

In [5]:
edf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1500 entries, 80056 to 54118
Data columns (total 15 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   StartTime  1500 non-null   object 
 1   Dur        1500 non-null   float64
 2   Proto      1500 non-null   object 
 3   SrcAddr    1500 non-null   object 
 4   Sport      1500 non-null   object 
 5   Dir        1500 non-null   object 
 6   DstAddr    1500 non-null   object 
 7   Dport      1500 non-null   object 
 8   State      1500 non-null   object 
 9   sTos       1491 non-null   float64
 10  dTos       1423 non-null   float64
 11  TotPkts    1500 non-null   float64
 12  TotBytes   1500 non-null   float64
 13  SrcBytes   1500 non-null   float64
 14  Label      1500 non-null   object 
dtypes: float64(6), object(9)
memory usage: 187.5+ KB


In [6]:
src, dst = 'to_node', 'from_node'

In [7]:
edf['to_node'] = edf.SrcAddr
edf['from_node'] = edf.DstAddr
edf['Node'] = range(len(edf))

In [8]:
edf

,StartTime,Dur,Proto,SrcAddr,Sport,Dir,DstAddr,Dport,State,sTos,dTos,TotPkts,TotBytes,SrcBytes,Label,to_node,from_node,Node
80056,2011-08-15 17:01:05.514132,0.000798,udp,111.255.28.147,54172,<->,147.32.84.229,13363,CON,0.0,0.0,2.0,313.0,64.0,flow=Background-UDP-Established,111.255.28.147,147.32.84.229,0
11048,2011-08-15 16:45:09.512155,0.000481,udp,87.240.219.1,25614,<->,147.32.84.229,13363,CON,0.0,0.0,2.0,242.0,180.0,flow=Background-UDP-Established,87.240.219.1,147.32.84.229,1
129236,2011-08-15 17:13:16.392492,0.001632,udp,147.32.86.50,60876,<->,147.32.80.9,53,CON,0.0,0.0,2.0,475.0,81.0,flow=To-Background-UDP-CVUT-DNS-Server,147.32.86.50,147.32.80.9,2
67971,2011-08-15 16:58:10.530850,2.488998,tcp,211.101.37.115,51607,->,147.32.84.2,21,SRPA_SPA,0.0,0.0,14.0,1311.0,511.0,flow=Background-TCP-Established,211.101.37.115,147.32.84.2,3
28386,2011-08-15 16:48:54.499351,0.000514,udp,216.185.73.130,62119,<->,147.32.84.229,13363,CON,0.0,0.0,2.0,133.0,73.0,flow=Background-UDP-Established,216.185.73.130,147.32.84.229,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123621,2011-08-15 17:11:42.141884,0.159994,udp,81.90.162.3,14329,<->,147.32.84.229,13363,CON,0.0,0.0,2.0,244.0,171.0,flow=Background-UDP-Established,81.90.162.3,147.32.84.229,1495
91688,2011-08-15 17:03:55.605044,0.001758,udp,74.104.118.124,13272,<->,147.32.84.229,13363,CON,0.0,0.0,2.0,525.0,465.0,flow=Background-UDP-Established,74.104.118.124,147.32.84.229,1496
105890,2011-08-15 17:07:13.356378,0.000236,udp,147.32.84.138,43256,<->,147.32.80.9,53,CON,0.0,0.0,2.0,214.0,81.0,flow=To-Background-UDP-CVUT-DNS-Server,147.32.84.138,147.32.80.9,1497
3868,2011-08-15 16:43:52.057938,43.145054,udp,41.230.241.2,16459,<->,147.32.84.229,13363,CON,0.0,0.0,6.0,1569.0,1389.0,flow=Background-UDP-Established,41.230.241.2,147.32.84.229,1498


In [9]:
good_cols = ['Dur', 'Proto', 'Sport',
       'Dport', 'State', 'TotPkts', 'TotBytes', 'SrcBytes',
       'Label', 'to_node', 'from_node']

good_cols_without_label = ['Dur', 'Proto', 'Sport',
       'Dport', 'State', 'TotPkts', 'TotBytes', 'SrcBytes',
        'to_node', 'from_node']

good_cols_without_label_or_edges = ['Dur', 'Proto', 'Sport',
       'Dport', 'State', 'TotPkts', 'TotBytes', 'SrcBytes']

hover_cols = ['Proto', 'Label', 'to_node', 'from_node']

In [10]:
g = graphistry.edges(edf, src, dst)

In [11]:
g2 = g.featurize(kind='edges', 
                 y = pd.DataFrame({'Label': edf.Label.values}), 
                 use_columns=good_cols_without_label, 
                use_scaler='minmax') # need positive feature values

#g2 = g.featurize(kind='edges', use_columns=good_cols_without_label)

[feature_utils.py:861 - get_dataframe_columns() ]
   returning DataFrame with columns `['Dur', 'Proto', 'Sport', 'Dport', 'State', 'TotPkts', 'TotBytes', 'SrcBytes', 'to_node', 'from_node']`

[feature_utils.py:158 - check_target_not_in_features() ]
   Removing [] columns from DataFrame

[feature_utils.py:738 - process_edge_dataframes() ]
   Encoding Edges using MultiLabelBinarizer

[feature_utils.py:741 - process_edge_dataframes() ]
   -Shape of Edge-2-Edge encoder (1500, 864)

[feature_utils.py:744 - process_edge_dataframes() ]
   -Rest of DataFrame has columns: Index(['Dur', 'Proto', 'Sport', 'Dport', 'State', 'TotPkts', 'TotBytes',
       'SrcBytes'],
      dtype='object') and is not empty

[feature_utils.py:495 -  get_textual_columns() ]
   No Textual Columns were found

[feature_utils.py:653 - process_dirty_dataframes() ]
   Encoding DataFrame might take a few minutes --------

[feature_utils.py:659 - process_dirty_dataframes() ]
   -Shape of data (1500, 135)


[feature_utils.py:6

In [12]:
g2.edge_features

,1.1.224.18,1.72.0.215,105.143.17.35,107.10.143.217,107.2.193.23,108.13.176.161,108.64.1.163,109.110.84.1,109.123.211.210,109.128.9.66,...,"State: con, s_, eco","State: pa_rpa, pa_r, sa_r","State: con, s_, ra_","State: spa_srpa, spa_sra, pa_r","State: con, s_, eco","State: con, s_, ra_",Dur,TotPkts,TotBytes,SrcBytes
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.00000,0.00528,1.00000,0.00333,1.00000,1.00000,0.00000,0.00024,0.00037,0.00021
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.00000,0.00528,1.00000,0.00333,1.00000,1.00000,0.00000,0.00024,0.00026,0.00058
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.00000,0.00528,1.00000,0.00333,1.00000,1.00000,0.00000,0.00024,0.00060,0.00026
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00484,0.00189,0.08014,0.01209,0.00075,0.04115,0.00138,0.00313,0.00181,0.00164
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.00000,0.00528,1.00000,0.00333,1.00000,1.00000,0.00000,0.00024,0.00011,0.00023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.00000,0.00528,1.00000,0.00333,1.00000,1.00000,0.00009,0.00024,0.00027,0.00055
1496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.00000,0.00528,1.00000,0.00333,1.00000,1.00000,0.00000,0.00024,0.00067,0.00149
1497,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.00000,0.00528,1.00000,0.00333,1.00000,1.00000,0.00000,0.00024,0.00022,0.00026
1498,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.00000,0.00528,1.00000,0.00333,1.00000,1.00000,0.02396,0.00120,0.00219,0.00446


In [47]:
g2.plot() # a big network topology graph

# After UMAP-ing, we get a pruned dataframe of implicit edges, below

In [14]:
g3 = g2.umap(kind='edges', metric='hellinger', n_neighbors=20, scale=0.1)

[umap_utils.py:92 -                  fit() ]
   Starting UMAP-ing data of shape (1500, 999)

OMP: Info #273: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
[umap_utils.py:105 -      _edge_influence() ]
   Calculating weighted adjacency (edge) DataFrame

[umap_utils.py:96 -                  fit() ]
   -UMAP-ing took 0.20 minutes total

[umap_utils.py:97 -                  fit() ]
    - or 7644.88 rows per minute

[feature_utils.py:829 - prune_weighted_edges_df_and_relabel_nodes() ]
   edge weights: mean(0.28), std(0.26), max(1.0), min(0.00), thresh(0.97)

[feature_utils.py:835 - prune_weighted_edges_df_and_relabel_nodes() ]
   Pruning weighted edge DataFrame from 37,700 to 2,434 edges.



In [15]:
g3.plot()

In [16]:
wdf = g3.weighted_edges_df_from_edges

In [17]:
coldict = dict(zip(range(len(g._edges)), g._edges['Label'].values))  # get the label dict per edge

In [18]:
cnt = Counter(g._edges['Label'])

In [19]:
cnt.most_common()

[('flow=To-Background-UDP-CVUT-DNS-Server', 589),
 ('flow=Background-UDP-Established', 448),
 ('flow=Background-TCP-Established', 164),
 ('flow=Background-Established-cmpgw-CVUT', 99),
 ('flow=Background', 40),
 ('flow=Background-UDP-Attempt', 36),
 ('flow=From-Normal-V46-Grill', 25),
 ('flow=From-Normal-V46-Stribrek', 23),
 ('flow=Background-TCP-Attempt', 20),
 ('flow=To-Background-CVUT-Proxy', 14),
 ('flow=From-Normal-V46-Jist', 12),
 ('flow=Background-Attempt-cmpgw-CVUT', 5),
 ('flow=From-Botnet-V46-TCP-Attempt', 4),
 ('flow=Background-UDP-NTP-Established-1', 4),
 ('flow=Background-google-analytics4', 3),
 ('flow=Background-google-analytics12', 2),
 ('flow=From-Background-CVUT-Proxy', 2),
 ('flow=Background-google-analytics1', 2),
 ('flow=To-Background-CVUT-WebServer', 1),
 ('flow=From-Botnet-V46-TCP-Not-Encrypted-SMTP-Private-Proxy-1', 1),
 ('flow=From-Botnet-V46-TCP-Established-HTTP-To-Microsoft-Live-1', 1),
 ('flow=Background-google-analytics15', 1),
 ('flow=From-Botnet-V46-TCP-E

In [29]:
src, dst = config.SRC, config.DST

In [30]:
wdf['src_label'] = wdf[src].apply(lambda x: coldict[x]) # map label for the given node as a sanity check
wdf['dst_label'] = wdf[dst].apply(lambda x: coldict[x])

In [22]:
wdf = wdf.fillna('')

In [23]:
hover_data = pd.DataFrame(g._edges.values, index=range(len(g._edges)), columns=g._edges.columns)
hover_data = hover_data.fillna('')

In [24]:
hover_data = hover_data[hover_cols+ ['Node']]

In [25]:
hover_data

,Proto,Label,to_node,from_node,Node
0,udp,flow=Background-UDP-Established,111.255.28.147,147.32.84.229,0
1,udp,flow=Background-UDP-Established,87.240.219.1,147.32.84.229,1
2,udp,flow=To-Background-UDP-CVUT-DNS-Server,147.32.86.50,147.32.80.9,2
3,tcp,flow=Background-TCP-Established,211.101.37.115,147.32.84.2,3
4,udp,flow=Background-UDP-Established,216.185.73.130,147.32.84.229,4
...,...,...,...,...,...
1495,udp,flow=Background-UDP-Established,81.90.162.3,147.32.84.229,1495
1496,udp,flow=Background-UDP-Established,74.104.118.124,147.32.84.229,1496
1497,udp,flow=To-Background-UDP-CVUT-DNS-Server,147.32.84.138,147.32.80.9,1497
1498,udp,flow=Background-UDP-Established,41.230.241.2,147.32.84.229,1498


In [31]:
g_2 = graphistry.edges(wdf, src, dst).nodes(hover_data, 'Node')
g_2.plot()

In [32]:
umap.plot.output_notebook()

Loading BokehJS ...

In [33]:
p = umap.plot.interactive(g3._umap, labels=edf['Label'], hover_data=hover_data, point_size=20, theme='fire')
umap.plot.show(p)

# Do we have a predictive model?

In [34]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
# use the SMOTE/OverSampler to balance out the classes
smote = SMOTE(random_state = 42)
ros = RandomOverSampler(random_state=0)


In [35]:
# let's turn the Label into a binary indicator to simplify things
T = edf.Label.apply(lambda x: 1 if 'Botnet' in x else 0)

In [36]:
T

80056     0
11048     0
129236    0
67971     0
28386     0
         ..
123621    0
91688     0
105890    0
3868      0
54118     0
Name: Label, Length: 1500, dtype: int64

In [37]:
T.sum()/T.shape[0]  ## very sparse label

0.005333333333333333

In [38]:
clf = RandomForestClassifier()
#X, y = ros.fit_resample(g.edge_features.values, T)
X, y = smote.fit_resample(g2.edge_features.values, T)

In [39]:
smote.sampling_strategy_

OrderedDict([(1, 1484)])

In [40]:
X.shape, y.shape

((2984, 999), (2984,))

In [41]:
y.sum()/y.shape[0] ## lots more data points from sampler

0.5

In [42]:
X_train, x_test, y_train, y_test = train_test_split(X, y) # standard splits

In [43]:
clf.fit(X_train, y_train)
clf.score(x_test, y_test) # huh? perfect ... likely due to it learning a single feature, like IP address

0.9986595174262735

In [44]:
fi = clf.feature_importances_.argsort()[::-1][:10]

In [45]:
mlb = g2.edge_encoders[0]

In [46]:
mlb.classes_[fi[0]] # so as expected, it has learned the IP address of the bot .. not any real predictive capacity

IndexError: index 941 is out of bounds for axis 0 with size 864

In [ ]:
n=len(mlb.classes_) # should be equal to total number of IP addresses
n

In [ ]:
dc = g2.edge_encoders[1]
feats = dc.get_feature_names_out()

In [ ]:
# most important features to predict target botnet...
[(feats[k-len(mlb.classes_)], clf.feature_importances_[k]) if k > n else (mlb.classes_[k], clf.feature_importances_[k]) for k in clf.feature_importances_.argsort()[::-1][:12]]

# Lets remove edges and see if there is a model of just 'common features' (ie no edges)

In [ ]:
# treat edf as ndf to featurize without specifying edges
g2 = g.nodes(edf).featurize(kind='nodes', use_columns=good_cols_without_label_or_edges, use_scaler='minmax') 

In [ ]:
X = g2.node_features

In [ ]:
use_cols = ['Proto', 'Sport']

In [ ]:
X

In [ ]:
cols = X.columns.values

In [ ]:
X.shape, T.shape

In [ ]:
Xp, yp = smote.fit_resample(X.values, T)

In [ ]:
yp.sum()/yp.shape[0]  # half the data is in class bot

In [ ]:
d = Xp[yp==1].std(0)/Xp[yp==0].std(0) + Xp[yp==1].mean(0)/Xp[yp==0].mean(0)
show = d > d.mean() + d.std()
fig = plt.figure(figsize=(10,5))
plt.plot(d)
z = plt.xticks(np.arange(len(d))[show], cols[show], rotation = 45)

In [ ]:
X_train, x_test, y_train, y_test = train_test_split(Xp, yp)

In [ ]:
clf.fit(X_train, y_train).score(x_test, y_test)  # so it def has figured something out that might be overfit...

In [ ]:
feats = g2.node_encoder.get_feature_names_out()

In [ ]:
thresh = clf.feature_importances_.mean() + 2* clf.feature_importances_.std()
thresh

In [ ]:
# let's see what most important features are?
[(feats[k], clf.feature_importances_[k]) for k in clf.feature_importances_.argsort()[::-1] if clf.feature_importances_[k]> thresh]

In [ ]:
res = [(feats[k], clf.feature_importances_[k]) for k in clf.feature_importances_.argsort()[::-1] if clf.feature_importances_[k]> thresh]

In [ ]:
fts = [k[0] for k in res]
fts

In [ ]:
X = g2.node_features

In [ ]:
X[T.values ==1][fts].describe() # bot features

In [ ]:
X[T.values ==0][fts].describe() # non bot mean features

In [ ]:
X.columns

## Shows a clear signature difference between bot and nonbot features

In [ ]:
nonBot = X[T.values ==0][fts].describe()
bot = X[T.values ==1][fts].describe()

In [ ]:
nonBot.loc['mean']/bot.loc['mean']

## We see that SrcBytes and Proto_tcp has much high means than the non bot traffic, likewise Proto_UDP and Duration are much lower than expected

In [ ]:
bot.loc['mean']/nonBot.loc['mean']

# Hence we see that including just common features clusters botnet traffic together under featurization and UMAP

In [ ]:
good_cols_without_label_or_edges # these are the features

In [ ]:
g4 = g2.umap(kind='nodes', scale = 1)

In [ ]:
g4.filter_edges(scale=2, inplace=True)

In [ ]:
wdf = g4.weighted_edges_df_from_nodes

In [ ]:
wdf

In [ ]:
g6 = graphistry.edges(wdf, '_src', '_dst').nodes(hover_data, 'Node')
g6.plot()

# Let's encode the graph as a DGL graph for use in Machine Learning

In [ ]:
src, dst = 'from_node', 'to_node'
node_column = 'Node'

# we instantiate a new feature mixin that also builds DGL graphs
#graph = DGLGraphMixin()

In [ ]:
# we can make an effective node_df using edf 
tdf = edf.groupby(['to_node'], as_index=False).mean().assign(ip=lambda x: x.to_node)
fdf = edf.groupby(['from_node'], as_index=False).mean().assign(ip=lambda x: x.from_node)
ndf = pd.concat([tdf, fdf], axis=0)
ndf = ndf.fillna(0)

node_cols = ['Dur', 'TotPkts', 'TotBytes', 'SrcBytes', 'ip']#, 'sTos', 'dTos']
use_cols = ['Dur', 'TotPkts', 'TotBytes', 'SrcBytes']
ndf = ndf[node_cols]
ndf = ndf.drop_duplicates(subset=['ip'])

In [ ]:
len(ndf.ip.unique())

In [ ]:
ndf

In [ ]:
assert len(ndf.ip.unique()) == len(ndf)

In [ ]:
y_edges=pd.DataFrame({'Label': edf.Label.values}, index=edf.index) # must include index or g._MASK will through error

In [ ]:
#TT = edf.Label.apply(lambda x: 'bot' if 'Botnet' in x else 'normal')
#y_edges = pd.DataFrame({'Label': TT}, index=edf.index)

In [ ]:
# now we can build the DGL graph with real data
g = graphistry.edges(edf, src, dst).nodes(ndf, 'ip')

In [ ]:
# g2=g.build_dgl_graph('ip', use_edge_columns=[src, dst], y_edges=y_edges, use_node_columns=node_cols)
# since we need the label
g2=g.build_dgl_graph('ip', 
                     use_edge_columns=good_cols_without_label, 
                     y_edges=y_edges, 
                     use_node_columns=use_cols, 
                    use_node_scaler='zscale', 
                    use_edge_scaler='minmax')
#g2=g.build_dgl_graph('ip', use_edge_columns=good_cols_without_label, use_node_columns=node_cols)

In [ ]:
# now we have a DGL graph with ndata and edata built via our featurization tools
g2.DGL_graph

In [ ]:
g.DGL_graph

In [ ]:
labels = None # in case we ran it without the label
if g2.edge_target is not None: # depends if we run line two above
    y_target=g2.edge_target.values.argmax(1)
    labels = y_target

In [ ]:
g5 = g2.umap(kind='edges', y=labels, scale=0.1)

In [ ]:
umap.plot.points(g5._umap, labels=labels, theme='fire')

In [ ]:
p = umap.plot.interactive(g5._umap, labels=labels, hover_data=hover_data, point_size=20, theme='fire')
umap.plot.show(p)  

# Now that we have this in hand, we can train a model

In [ ]:
from graphistry.networks import LinkPredModelMultiOutput

In [ ]:
# get the DGL graph object
G = g.DGL_graph

In [ ]:
G

In [ ]:
# we have to allow self loops if it gives error below, but seems to work without!
# G = dgl.add_self_loop(G)

In [ ]:
# to get a sense of the different parts in training loop above
features = G.ndata['feature']
labels = G.edata['target']
#labels = torch.tensor(T.values, dtype=torch.float)
train_mask = G.edata['train_mask']
test_mask = G.edata['test_mask']
targets = labels.argmax(1)

In [ ]:
targets

In [ ]:
labels

In [ ]:
np.array(labels)

# Define the Model 

In [ ]:
n_feat = G.ndata['feature'].shape[1]
latent_dim = 32
n_output_feats = 16 # this is equal to the latent dim output of the SAGE net, not n_targets

node_features = G.ndata['feature'].float()
edge_label = G.edata['target']

n_targets = edge_label.shape[1]
targets = edge_label.argmax(1)

train_mask = G.edata['train_mask']

In [ ]:
# instantiate model
model = LinkPredModelMultiOutput(n_feat, latent_dim, n_output_feats, n_targets) # 1) #LinkPredModel(n_feat, latent_dim, n_output_feats)

pred = model(G, node_features)

print(f'output of model should have same length as the number of edges: {pred.shape[0]}')
print(f'number of edges: {G.num_edges()}')
assert G.num_edges() == pred.shape[0], 'something went wrong'

In [ ]:
pred # untrained predictor

## Train the Model

In [ ]:
import torch
import torch.nn.functional as F

In [ ]:
# the optimizer does all the backprop
opt = torch.optim.Adam(model.parameters())

use_cross_entropy_loss = True

for epoch in range(2000):
    logits = model(G, node_features)
    
    if use_cross_entropy_loss:    
        loss = F.cross_entropy(logits[train_mask], edge_label[train_mask])
    else:
        loss = ((logits[train_mask] - edge_label[train_mask]) ** 2).mean()

    opt.zero_grad()
    loss.backward()
    opt.step()
    if epoch%100==0:
        print(loss.item())

In [ ]:
# trained comparison
logits = model(G, node_features)
pred = logits.argmax(1)

accuracy = sum(pred[test_mask] == targets[test_mask])/len(pred[test_mask]) # does well! 
print(f'Accuracy: {100*accuracy:.2f}%')
# this should not impress us too much as most of the labels are from one 'category'

In [ ]:
pred[targets!=58]

In [ ]:
targets[targets!=58]

In [ ]:
accuracy = sum(pred[targets!=58] == targets[targets!=58])/len(pred[targets!=58]) # does well! 
print(f'Accuracy: {100*accuracy:.2f}%')
# this should not impress us too much as most of the labels are from one 'category'

In [ ]:
from sklearn.metrics import confusion_matrix, plot_confusion_matrix

M = confusion_matrix(pred, targets)

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

In [ ]:
# so we see that the imbalance classes aren't doing great in this example. Should we be surprised? 
# Most of the labels are in the first three classes below

In [ ]:
cnt = Counter(list(pred.tolist()))
cnt.most_common()[:10]

In [ ]:
## here are the botnet labels in the GNN, very few considering how big the dataset is
np.array(targets)[T==1]

In [ ]:
np.array(pred)[T==1] # the model does a good job at finding botnet...

In [ ]:
# however, the model does a good job at finding botnet...but we shouldn't be surprised given the graph structure.
ConfusionMatrixDisplay.from_predictions(np.array(pred)[(T==1) & np.array(test_mask)], np.array(targets)[(T==1) & np.array(test_mask)])



In [ ]:
ConfusionMatrixDisplay.from_predictions(np.array(pred)[T==1], np.array(targets)[T==1])

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
from pprint import pprint

In [ ]:
named_labels = [g]

In [ ]:
print(classification_report(targets, pred, target_names=labels))

# Now we can inspect the different layer activations

In [ ]:
# To get forward activations 
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

In [ ]:
model.sage.conv1.register_forward_hook(get_activation('conv1'))
model.sage.conv2.register_forward_hook(get_activation('conv2'))
model.pred.W.register_forward_hook(get_activation('W'))
# now you call model 
logits = model(G, features.float())
# this will load the dictionary
print(activation['conv1'])
print(activation['conv2'])
print(activation['W'])

In [ ]:
activation['conv1'].shape  # same as number of nodes!

In [ ]:
activation['conv2'].shape

In [ ]:
activation['W'].shape

In [ ]:
plt.figure(); plt.imshow(activation['conv2'], aspect='auto',  cmap=plt.get_cmap('plasma'))

In [ ]:
# Now we can use the activations from the SAGE conv as node features

In [ ]:
enc_features = np.array(activation['conv2'])

In [ ]:
enc_features.shape

In [ ]:
# Can we go from edge target (botnet) backwards to node features??

In [ ]:
botedf = edf.loc[T==1]

In [ ]:
botedf.to_node.unique() # the bad IP address

In [ ]:
botedf.from_node.unique() # the affected devices

In [ ]:
ndf[ndf.ip=='147.32.84.165'].mean()

In [ ]:
ndf[ndf.ip!='147.32.84.165'].mean()

In [ ]:
ndf[ndf.ip.isin(botedf.from_node.unique())].mean()/ndf.mean()

In [ ]:
ndf[ndf.ip.isin(botedf.to_node.unique())].mean()/ndf.mean()

In [ ]:
bot = enc_features[ndf.ip=='147.32.84.165']

In [ ]:
notbot = enc_features[ndf.ip!='147.32.84.165'].mean(0)

In [ ]:
 # needles in logstacks ?

In [ ]:
plt.plot(bot[0])
plt.plot(notbot)

In [ ]:
g8 = g.umap(X=enc_features, kind='nodes')

In [ ]:
g8.node_embedding

In [ ]:
g8.node_embedding.shape

In [ ]:
labels2 = np.zeros(len(ndf))
labels2[ndf.ip=='147.32.84.165'] = 1 

In [ ]:
g._edges.shape

In [ ]:
sum(labels2==1)

In [ ]:
p = umap.plot.interactive(g8._umap, labels=labels2, hover_data=hover_data, point_size=20, theme='fire')
umap.plot.show(p)  